# Model Comparison Scripts

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import seaborn as sns
import json

from datetime import datetime
from PIL import Image
from statistics import mean
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [2]:
# Homemade Utility Classes
from pytorch_vision_utils.Utilities import clear_dirs, time_to_predict, DataVisualizationUtilities, TrainingUtilities

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using: ", device)

Using:  cuda:0


In [4]:
# DIRECTORY NAMES
with open("parameters.json", "r") as f:
    print("Loading parameters...\n")
    params = dict(json.load(f))
    
    DATA_DIR = params["DATA_DIR"]
    TEST_DIR = params["TEST_DIR"]
    MODEL_DIR = params["MODEL_DIR"]
    MEDIA_DIR = params["MEDIA_DIR"]
    INC_DIR = params["INC_DIR"]
    
    print("Loading parameters complete!")
    print(MODEL_DIR)
    print(TEST_DIR)
    print(MEDIA_DIR)
    print(DATA_DIR)
    
MODEL_NAME1 = "mobilenetv2"
MODEL_NAME2 = "xception"
CONSTANT = 1.5


Loading parameters...

Loading parameters complete!
saved_models/
validation/
media/
train/


# Create models for Comparison

In [5]:
%run -m pytorch_vision_utils.train -- --model_name "{MODEL_NAME1}" --parameters_path "parameters.json" --debug "True"

Using:  cuda
{'BATCH_SIZE': 16, 'ETA': 0.0001, 'PATIENCE': 3, 'CROP_SIZE': 224, 'DEGREES': 20, 'HUE': 0.3, 'BRIGHTNESS': 0.2, 'SATURATION': 0.2, 'CONTRAST': 0.2, 'MONITOR': 'val_loss', 'MIN_DELTA': 0, 'LR_PATIENCE': 5, 'FACTOR': 0.5, 'INPUT_SIZE': [224, 224], 'MEAN': [0.485, 0.456, 0.406], 'STD': [0.229, 0.224, 0.225]}
MobileNetV2Wrapper(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, 

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

C:\Users\nicol\Anaconda3\envs\torch_base\lib\site-packages\IPython\core\interactiveshell.py:2838: UserWarning: Unknown failure executing module: <pytorch_vision_utils.train>
  warn('Unknown failure executing module: <%s>' % mod_name)


In [ ]:
%run -m pytorch_vision_utils.train -- --model_name "${MODEL_NAME2}" --parameters_path "parameters.json" --debug "True"

In [ ]:
train_utils = TrainingUtilities(data_dir=TEST_DIR, model_dir=MODEL_DIR, 
                                model_name=MODEL_NAME1, mode="test")
dataviz_utils = DataVisualizationUtilities()
clear_dirs(INC_DIR)

In [ ]:
train_utils.load_weights(MODEL_NAME1, MODEL_DIR+"/"+MODEL_NAME1+".pth")
dataviz_utils.display_dataset(train_utils)

## Model 1 Results

In [ ]:
dataviz_utils.display_metric_results(0, train_utils, img_dir=INC_DIR)

In [ ]:
dataviz_utils.display_roc_curve(0, train_utils)

In [ ]:
model_name1_times = time_to_predict(train_utils.model, train_utils.loader, constant=CONSTANT)

## Model 2 Results

In [ ]:
train_utils.load_weights(MODEL_NAME2, MODEL_DIR+"/"+MODEL_NAME2+".pth")
dataviz_utils.display_metric_results(0, train_utils, img_dir=INC_DIR)

In [ ]:
dataviz_utils.display_roc_curve(0, train_utils)

In [ ]:
model_name2_times = time_to_predict(train_utils.model, train_utils.loader, constant=CONSTANT)

## Prediction Benchmark Graph

In [ ]:
dataviz_utils.display_benchmark_results(model_name1_times, model_name2_times, model_name1=MODEL_NAME1, model_name2=MODEL_NAME2, bw_adjust=3)